# LLM Workshop: LangGraph with Google Gemini

This workshop covers:
- Introduction to Google Gemini
- LangGraph with Gemini integration
- Prompting techniques with Gemini
- System prompts and personas
- Building workflows with Gemini

## Setup and Installation

In [2]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import Graph, StateGraph
from typing import TypedDict, List
from IPython.display import Markdown, display

# Load environment variables
load_dotenv()

# Initialize Gemini LLM
# Make sure to set GOOGLE_API_KEY in your .env file
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.7,
    max_tokens=100
)

E0000 00:00:1758154839.780101 3954261 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


### Environment Setup

Make sure to create a `.env` file with your Google API key:
```
GOOGLE_API_KEY=your_google_api_key_here
```

You can get your API key from: https://makersuite.google.com/app/apikey

## 1. Basic Gemini Interaction

In [3]:
# Simple Gemini call
response = llm.invoke([HumanMessage(content="What is artificial intelligence?")])
display(Markdown(response.content))

Artificial intelligence (AI) is a broad field encompassing the theory and development of computer systems able to perform tasks that normally require human intelligence.  This includes tasks like:

* **Learning:** Acquiring information and rules for using the information.
* **Reasoning:** Using rules to reach approximate or definite conclusions.
* **Problem-solving:** Finding solutions to complex situations.
* **Perception:** Interpreting sensory information like images, sound, and text.
* **Language understanding:**  Processing

## 2. Understanding System Prompts with Gemini

System prompts define the AI's behavior and personality. Gemini handles system messages effectively.

In [ ]:
# Example with system prompt
messages = [
    SystemMessage(content="You are a helpful coding assistant specializing in Python. Always provide clear, concise explanations with practical code examples."),
    HumanMessage(content="Explain what a Python dictionary is")
]

response = llm.invoke(messages)
display(Markdown(response.content))

### Exercise 1: Create different personas with system prompts

In [ ]:
# Try different system prompts with Gemini
personas = {
    "teacher": "You are a patient university professor explaining complex concepts to undergraduate students. Use analogies and examples.",
    "expert": "You are a senior AI researcher providing detailed technical analysis with academic rigor.",
    "creative": "You are an innovative storyteller who explains concepts through engaging narratives and metaphors."
}

question = "What is deep learning?"

for persona, system_prompt in personas.items():
    print(f"\n=== {persona.upper()} RESPONSE ===")
    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=question)
    ]
    response = llm.invoke(messages)
    display(Markdown(response.content[:250] + "..."))

## 3. Prompting Techniques with Gemini

### 3.1 Zero-shot Prompting

In [ ]:
# Zero-shot: Direct question without examples
prompt = "Classify the sentiment of this text as Positive, Negative, or Neutral: 'I absolutely love this new feature!'"
response = llm.invoke([HumanMessage(content=prompt)])
display(Markdown(f"Zero-shot result: {response.content}"))

### 3.2 Few-shot Prompting

In [ ]:
# Few-shot: Provide examples for Gemini
few_shot_prompt = """
Classify the sentiment of the following texts as Positive, Negative, or Neutral:

Text: "This application is fantastic and user-friendly!"
Sentiment: Positive

Text: "The service was terrible and slow."
Sentiment: Negative

Text: "The weather is normal today."
Sentiment: Neutral

Text: "I absolutely love this new feature!"
Sentiment:
"""

response = llm.invoke([HumanMessage(content=few_shot_prompt)])
display(Markdown(f"Few-shot result: {response.content}"))

### 3.3 Chain-of-Thought Prompting

In [ ]:
# Chain-of-thought: Ask Gemini for step-by-step reasoning
cot_prompt = """
Solve this step by step:

A coffee shop has 20 tables. Each table can seat 3 people. 
If the coffee shop is 75% full, how many people are currently seated?

Think through this step by step and show your work:
"""

response = llm.invoke([HumanMessage(content=cot_prompt)])
display(Markdown(response.content))

In [ ]:
# Increase token limit for more detailed responses
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.7,
    max_tokens=500
)

response = llm.invoke([HumanMessage(content=cot_prompt)])
display(Markdown(response.content))

## 4. Introduction to LangGraph with Gemini

LangGraph helps build stateful, multi-step workflows with Gemini as the underlying LLM.

In [ ]:
# Define state for our graph
class GraphState(TypedDict):
    messages: List[str]
    current_step: str
    result: str

### 4.1 Simple Linear Workflow with Gemini

In [ ]:
def analyze_text(state: GraphState) -> GraphState:
    """Analyze the input text using Gemini"""
    text = state["messages"][-1]
    
    prompt = f"""
    Analyze the following text and provide a structured analysis with:
    1. Main topic
    2. Sentiment analysis
    3. Key themes and concepts
    4. Writing style assessment
    
    Text: {text}
    """
    
    response = llm.invoke([HumanMessage(content=prompt)])
    
    state["current_step"] = "analysis"
    state["result"] = response.content
    
    return state

def summarize_analysis(state: GraphState) -> GraphState:
    """Create a summary of the analysis using Gemini"""
    analysis = state["result"]
    
    prompt = f"""
    Create a concise executive summary of this analysis in 2-3 sentences.
    Focus on the most important insights:
    
    {analysis}
    """
    
    response = llm.invoke([HumanMessage(content=prompt)])
    
    state["current_step"] = "summary"
    state["result"] = response.content
    
    return state

In [ ]:
# Build the graph with Gemini
workflow = StateGraph(GraphState)

# Add nodes
workflow.add_node("analyze", analyze_text)
workflow.add_node("summarize", summarize_analysis)

# Add edges
workflow.add_edge("analyze", "summarize")

# Set entry point
workflow.set_entry_point("analyze")
workflow.set_finish_point("summarize")

# Compile the graph
app = workflow.compile()

In [ ]:
# Visualize the workflow
from IPython.display import Image

Image(app.get_graph().draw_mermaid_png())

In [ ]:
# Test the workflow with Gemini
initial_state = {
    "messages": ["Google's Gemini represents a significant breakthrough in multimodal AI capabilities. This advanced language model can process and understand text, images, and code simultaneously, opening new possibilities for creative problem-solving and innovative applications across various industries."],
    "current_step": "start",
    "result": ""
}

result = app.invoke(initial_state)
print("Final Summary:")
display(Markdown(result["result"]))

## 5. Building a Conditional Workflow with Gemini

In [ ]:
def classify_query(state: GraphState) -> GraphState:
    """Classify the type of query using Gemini"""
    query = state["messages"][-1]
    
    prompt = f"""
    Classify this query into exactly one of these categories:
    - technical: Programming, coding, software development, or technical questions
    - creative: Creative writing, brainstorming, artistic requests, or storytelling
    - factual: Factual questions, information requests, or knowledge queries
    
    Query: {query}
    
    Respond with only the category name (technical, creative, or factual).
    """
    
    response = llm.invoke([HumanMessage(content=prompt)])
    classification = response.content.strip().lower()
    
    state["current_step"] = "classified"
    state["result"] = classification
    
    return state

def handle_technical(state: GraphState) -> GraphState:
    """Handle technical queries with Gemini"""
    query = state["messages"][-1]
    
    system_prompt = "You are a senior software engineer with expertise in multiple programming languages. Provide detailed technical explanations with practical code examples and best practices."
    
    response = llm.invoke([
        SystemMessage(content=system_prompt),
        HumanMessage(content=query)
    ])
    
    state["current_step"] = "technical_response"
    state["result"] = response.content
    
    return state

def handle_creative(state: GraphState) -> GraphState:
    """Handle creative queries with Gemini"""
    query = state["messages"][-1]
    
    system_prompt = "You are a creative writing assistant and brainstorming expert. Be imaginative, inspiring, and help users explore innovative ideas with vivid descriptions and engaging narratives."
    
    response = llm.invoke([
        SystemMessage(content=system_prompt),
        HumanMessage(content=query)
    ])
    
    state["current_step"] = "creative_response"
    state["result"] = response.content
    
    return state

def handle_factual(state: GraphState) -> GraphState:
    """Handle factual queries with Gemini"""
    query = state["messages"][-1]
    
    system_prompt = "You are a knowledgeable research assistant. Provide accurate, well-structured information with relevant context and reliable details. Always strive for clarity and completeness."
    
    response = llm.invoke([
        SystemMessage(content=system_prompt),
        HumanMessage(content=query)
    ])
    
    state["current_step"] = "factual_response"
    state["result"] = response.content
    
    return state

def route_query(state: GraphState) -> str:
    """Route to appropriate handler based on Gemini's classification"""
    classification = state["result"]
    
    if "technical" in classification:
        return "technical"
    elif "creative" in classification:
        return "creative"
    else:
        return "factual"

In [ ]:
# Build conditional workflow with Gemini
conditional_workflow = StateGraph(GraphState)

# Add nodes
conditional_workflow.add_node("classify", classify_query)
conditional_workflow.add_node("technical", handle_technical)
conditional_workflow.add_node("creative", handle_creative)
conditional_workflow.add_node("factual", handle_factual)

# Add conditional edges
conditional_workflow.add_conditional_edges(
    "classify",
    route_query,
    {
        "technical": "technical",
        "creative": "creative",
        "factual": "factual"
    }
)

# Set entry and finish points
conditional_workflow.set_entry_point("classify")
conditional_workflow.set_finish_point("technical")
conditional_workflow.set_finish_point("creative")
conditional_workflow.set_finish_point("factual")

# Compile
conditional_app = conditional_workflow.compile()

In [ ]:
Image(conditional_app.get_graph().draw_mermaid_png())

### Test the conditional workflow with Gemini

In [ ]:
# Test with different query types using Gemini
test_queries = [
    "How do I implement a recursive function to calculate Fibonacci numbers in Python?",
    "Write a creative short story about an AI that discovers it can paint emotions",
    "What are the main differences between supervised and unsupervised machine learning?"
]

for query in test_queries:
    print(f"\n=== QUERY: {query} ===")
    
    state = {
        "messages": [query],
        "current_step": "start",
        "result": ""
    }
    
    result = conditional_app.invoke(state)
    print(f"Classification: {result['current_step']}")
    display(Markdown(f"Response: {result['result'][:300]}..."))

## 6. Exploring Gemini-Specific Features

Gemini offers unique capabilities that can enhance your LangGraph workflows.

In [ ]:
# Example: Using different Gemini models
gemini_pro = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0.3,
    max_tokens=200
)

# Compare responses between models
complex_query = "Explain the concept of quantum entanglement and its implications for quantum computing"

print("=== Gemini Flash Response ===")
flash_response = llm.invoke([HumanMessage(content=complex_query)])
display(Markdown(flash_response.content[:300] + "..."))

print("\n=== Gemini Pro Response ===")
pro_response = gemini_pro.invoke([HumanMessage(content=complex_query)])
display(Markdown(pro_response.content[:300] + "..."))